In [ ]:
# Installing Necessary Packages
!pip install PyMySQL
!pip install mysql-connector-python
!pip install sqlalchemy
import os
import pymysql
import mysql.connector
from sqlalchemy import create_engine
import datetime
import pymongo
import pprint
import json

import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import numpy as np

In [ ]:
# Specifying/Loading ID's
host_name = "localhost"
host_ip = "127.0.0.1"
port = "3306"

user_id = "ds2002"
pwd = "UVA!1819"
db_name = "project1"

In [ ]:
# Connecting to MySQL 
try: 
    conn = pymysql.connect(host=host_name, user=user_id, password=pwd, database=db_name)
    cursor = conn.cursor()
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600) # creating SQLAlchemy Object 
    connection = sqlEngine.connect() # connects to MySQL database using SQLAlchemy Object

    conn.close()
except Exception as e: 
    print(f"Error connecting to database with SQLAlchemy Object")

In [ ]:
# Reading Data from MySQL Database
try: 
    dim_table1 = pd.read_sql('SELECT * FROM dim_airlines', con=sqlEngine)
    dim_table2 = pd.read_sql('SELECT * FROM dim_planes', con=sqlEngine)
    dim_table3 = pd.read_sql('SELECT * FROM dim_weather', con=sqlEngine)
    dim_table1['index'] = range(len(dim_table1))
except as Exception as e: 
    print(f"Error reading data from database")

In [ ]:
# Merging 3 DataFrames Together into 1
try: 
    fact_table = pd.merge(dim_table1, dim_table2, on='index').merge(dim_table3, on='index')
except as Exception as e: 
    print(f"Error merging dataframes")

In [27]:
# Adding 3 Columns to 'fact_table' DataFrame and Performing Aggregation Queries
try: 
    fact_table['passengerTotal'] = fact_table['Passenger Count'].sum() # total number of passengers
    fact_table['passengerMean'] = fact_table['Passenger Count'].mean() # mean number of passengers
    fact_table['passengerMax'] = fact_table['Passenger Count'].max() # maximum number of passengers
except as Exception as e: 
    print(f"Error creating columns and inserting into fact_table")

In [28]:
# Saving 'fact_table' DataFrame to SQL Database Called 'fact_table'
try: 
    fact_table.to_sql('fact_table', con=sqlEngine, if_exists='replace', index=False)
    fact_table.head()
except Exception as e: 
    print(f"Error writing data to database")

,Activity Period,Operating Airline,Operating Airline IATA Code,Published Airline,Published Airline IATA Code,GEO Summary,GEO Region,Activity Type Code,Price Category Code,Passenger Count,...,skyConditionObservation,presentWeatherObservationManual,skyCoverLayer,atmosphericPressureObservation,liquidPrecipitation,extremeAirTemperature,passengerTotal,passengerMean,statusCount,passengerMax
0,200507,ATA Airlines,TZ,ATA Airlines,TZ,Domestic,US,Deplaned,Low Fare,27271,...,nan,nan,nan,nan,nan,nan,3409770,26638.828125,128,443656
1,200507,ATA Airlines,TZ,ATA Airlines,TZ,Domestic,US,Enplaned,Low Fare,29131,...,"{'totalCoverage': {'value': '07', 'opaque': '9...","[{'condition': '02', 'quality': '1'}]",nan,nan,nan,nan,3409770,26638.828125,128,443656
2,200507,ATA Airlines,TZ,ATA Airlines,TZ,Domestic,US,Thru / Transit,Low Fare,5415,...,"{'totalCoverage': {'value': '06', 'opaque': '9...","[{'condition': '26', 'quality': '1'}]",nan,nan,nan,nan,3409770,26638.828125,128,443656
3,200507,Air Canada,AC,Air Canada,AC,International,Canada,Deplaned,Other,35156,...,nan,nan,nan,nan,nan,nan,3409770,26638.828125,128,443656
4,200507,Air Canada,AC,Air Canada,AC,International,Canada,Enplaned,Other,34090,...,nan,nan,nan,nan,nan,nan,3409770,26638.828125,128,443656


In [ ]:
# This file creates a fact_table using the data loaded from MongoDB, SQL, and a CSV (3 dimensions). 
# 3 aggregation queries are performed to see the total, mean, and maximum number of passengers based on information pulled and inserted into the new data frame.